# 유튜브 댓글 크롤링

In [1]:
from selenium import webdriver
import time
from openpyxl import Workbook
import pandas as pd
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from konlpy.tag import Okt

#한글 댓글에 대한 단어 처리
okt = Okt()

# 워크북 선언

In [ ]:
wb = Workbook(write_only=True)
ws = wb.create_sheet("sheet1",1)

# 크롬 드라이버 사용

In [ ]:
driver = webdriver.Chrome("C:/tool/chromedriver.exe")

# 영상 URL & 스크롤 위치 초기화

In [ ]:
driver.get("https://www.youtube.com/watch?v=xRbPAVnqtcs")
driver.implicitly_wait(3)

time.sleep(1.5)

driver.execute_script("window.scrollTo(0, 800)")
time.sleep(3)

# 댓글 페이지 추가 노출 코드

In [ ]:
last_height = driver.execute_script("return document.documentElement.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(1.5)

    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

time.sleep(1.5)

# 유튜브 팝업창 닫기

In [ ]:
try:
    driver.find_element_by_css_selector("#dismiss-button > a").click()
except:
    pass

# 대댓글까지 추가로 노출 코드(선택)

In [ ]:
#buttons = driver.find_elements_by_css_selector("#more-replies > a")

#time.sleep(1.5)

#for button in buttons:
#    button.send_keys(Keys.ENTER)
#    time.sleep(1.5)
#   button.click()

# 댓글 크롤링

In [ ]:
html_source = driver.page_source
soup = BeautifulSoup(html_source, 'html.parser')

id_list = soup.select("div#header-author > h3 > #author-text > span")
comment_list = soup.select("yt-formatted-string#content-text")

id_final = []
comment_final = []

for i in range(len(comment_list)):
    temp_id = id_list[i].text
    temp_id = temp_id.replace('\n', '')
    temp_id = temp_id.replace('\t', '')
    temp_id = temp_id.replace('    ', '')
    id_final.append(temp_id)

    temp_comment = comment_list[i].text
    temp_comment = temp_comment.replace('\n', '')
    temp_comment = temp_comment.replace('\t', '')
    temp_comment = temp_comment.replace('    ', '')
    print(temp_comment)
    comment_final.append(temp_comment)
    

 

# 데이터 정제

In [ ]:
keyword_list = []
word = ""
for i in range(len(comment_list)):
    
    words = okt.nouns(comment_final[i])
   
    
    word += " ".join(words)
    word += " "
print(word[:10])

# 정제된 데이터 빈도수 분석

In [ ]:
wordList = word.split()
wordLists = []
wordCount = {} 
wordCounts = []
 # word -> count 기준 정렬한 dictionary
for word in wordList:
    # Get 명령어를 통해, Dictionary에 Key가 없으면 0리턴
    wordCount[word] = wordCount.get(word, 0) + 1 
    
keys = sorted(wordCount.items(),key=(lambda x:x[1]),reverse=True)

for word in keys:
    print(word[0] + ':' + str(word[1])) 
    wordLists.append(word[0])  
    wordCounts.append(word[1])

# 댓글 내용 데이터 저장(Excel)

In [ ]:
pd_data = {"아이디" : id_final , "댓글 내용" : comment_final}
youtube_pd = pd.DataFrame(pd_data)

youtube_pd.to_excel('youtube_comment.xlsx')

# 댓글 빈도수 분석 데이터 저장(Excel)

In [ ]:
pd2_data = {"키워드" : wordLists, "빈도수" : wordCounts}
youtube_pd2 = pd.DataFrame(pd2_data)

youtube_pd2.to_excel('youtube_keyword_count.xlsx')